In [51]:
from ibl_pipeline import acquisition, behavior, ephys, subject
from ibl_pipeline.analyses import behavior as behavior_analyses
import datajoint as dj
import pandas as pd

from oneibl.one import ONE
one = ONE()

Connected to https://alyx.internationalbrainlab.org as anneu


In [60]:
# grab insertions and behavioral QC from datajoint
sessions = acquisition.Session & (acquisition.SessionProject & 'session_project like "%brainwide%"') & \
    ephys.DefaultCluster()
insertions_qc = subject.SubjectLab * ephys.ProbeInsertion * sessions * (behavior_analyses.SessionTrainingStatus)
insertions_qc = insertions_qc.proj('session_uuid', 'session_lab', behavior_qc_passed='good_enough_for_brainwide_map')
insertions = insertions_qc.fetch(format='frame').reset_index()


In [61]:
insertions['alyx_qc'] = 'empty'
for eid, dat in insertions.groupby(['session_uuid']):
    # bit awkward, have to extract string from UUID object
    sess_info = one.alyx.rest('sessions', 'read', id=eid.urn[9:])
    insertions.loc[insertions.session_uuid == eid, 'alyx_qc'] = sess_info['qc']
    print(eid, ': ', sess_info['qc'])


01864d6f-31e8-49c9-aadd-2e5021ea0ee7 :  NOT_SET
02fbb6da-3034-47d6-a61b-7d06c796a830 :  NOT_SET
032ffcdf-7692-40b3-b9ff-8def1fc18b2e :  NOT_SET
034e726f-b35f-41e0-8d6c-a22cc32391fb :  NOT_SET
03cf52f6-fba6-4743-a42e-dd1ac3072343 :  NOT_SET
08e25a6c-4223-462a-a856-a3a587146668 :  NOT_SET
097afc11-4214-4879-bd7a-643a4d16396e :  NOT_SET
097ba865-f424-49a3-96fb-863506fac3e0 :  NOT_SET
0cbeae00-e229-4b7d-bdcc-1b0569d7e0c3 :  NOT_SET
0deb75fb-9088-42d9-b744-012fb8fc4afb :  NOT_SET
0f25376f-2b78-4ddc-8c39-b6cdbe7bf5b9 :  NOT_SET
1120797f-c2b0-4c09-b6ea-2555d69cb7ee :  NOT_SET
115d264b-1939-4b8e-9d17-4ed8dfc4fadd :  NOT_SET
12dc8b34-b18e-4cdd-90a9-da134a9be79c :  NOT_SET
1538493d-226a-46f7-b428-59ce5f43f0f9 :  NOT_SET
158d5d35-a2ab-4a76-87b0-51048c5d5283 :  NOT_SET
15f742e1-1043-45c9-9504-f1e8a53c1744 :  NOT_SET
16693458-0801-4d35-a3f1-9115c7e5acfd :  NOT_SET
16c3667b-e0ea-43fb-9ad4-8dcd1e6c40e1 :  NOT_SET
1c213d82-32c3-49f7-92ca-06e28907e1b4 :  NOT_SET
1c27fd32-e872-4284-b9a5-7079453f4cbc :  

In [65]:
# SUMMARIZE PER LAB/USER
insertions.groupby(['alyx_qc', 'behavior_qc_passed', 'session_lab'])['session_start_time'].count()

alyx_qc  behavior_qc_passed  session_lab   
NOT_SET  0                   angelakilab        7
                             churchlandlab     16
                             cortexlab          9
                             danlab            12
                             mainenlab          1
                             mrsicflogellab    30
                             wittenlab          1
                             zadorlab          11
         1                   angelakilab       15
                             churchlandlab     69
                             cortexlab         30
                             danlab            38
                             mainenlab          2
                             mrsicflogellab    12
                             wittenlab          4
                             zadorlab          27
Name: session_start_time, dtype: int64

In [66]:
# SHOW EACH INSERTION WITH ITS QC VALUES
pd.set_option('display.max_rows', 1000)
insertions

,subject_uuid,session_start_time,probe_idx,session_uuid,session_lab,behavior_qc_passed,alyx_qc
0,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-17 15:55:29,0,01864d6f-31e8-49c9-aadd-2e5021ea0ee7,angelakilab,1,NOT_SET
1,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-18 14:25:54,0,6713a4a7-faed-4df2-acab-ee4e63326f8d,angelakilab,1,NOT_SET
2,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-18 14:25:54,1,6713a4a7-faed-4df2-acab-ee4e63326f8d,angelakilab,1,NOT_SET
3,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-19 15:08:36,0,dd0faa76-4f49-428c-9507-6de7382a5d9e,angelakilab,0,NOT_SET
4,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-20 15:29:44,0,266a0360-ea0a-4580-8f6a-fe5bad9ed17c,angelakilab,0,NOT_SET
5,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-20 15:29:44,1,266a0360-ea0a-4580-8f6a-fe5bad9ed17c,angelakilab,0,NOT_SET
6,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-21 15:25:03,0,56956777-dca5-468c-87cb-78150432cc57,angelakilab,1,NOT_SET
7,0f47b24d-3909-428c-a18d-666006c7c834,2020-02-21 15:25:03,1,56956777-dca5-468c-87cb-78150432cc57,angelakilab,1,NOT_SET
8,15f98ab5-20d8-4345-a5de-21912a39575f,2020-01-20 14:56:51,0,89f0d6ff-69f4-45bc-b89e-72868abb042a,churchlandlab,1,NOT_SET
9,15f98ab5-20d8-4345-a5de-21912a39575f,2020-01-21 13:52:24,0,b52182e7-39f6-4914-9717-136db589706e,churchlandlab,1,NOT_SET
